In [ ]:
import json
import urllib.request
import torch
from torchvision import models, transforms
from PIL import Image

# ImageNetのクラス名をダウンロードします
class_idx = json.load(urllib.request.urlopen('https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'))

# 学習済みのResNet50モデルをロードします
resnet = models.resnet50(pretrained=True)
resnet.eval()

# 入力画像を処理するための変換を定義します
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

while True:
    # ユーザーに画像URLを入力させます
    img_url = input("画像のURLを入力してください（終了するには'q'を入力）：")

    if img_url.lower() == 'q':
        break

    # URLから画像をダウンロードし、メモリ内で開きます
    with urllib.request.urlopen(img_url) as url:
        img = Image.open(url)

    # 画像を変換します
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)

    # モデルを使って画像を推測します
    resnet.eval()
    out = resnet(batch_t)

    # ソフトマックス関数を適用して確率を計算します
    probs = torch.nn.functional.softmax(out, dim=1)[0] * 100

    # 上位3つの予測結果を取得します
    _, indices = torch.topk(probs, 3)
    percentages = probs[indices]

    # 推測結果を表示します
    for idx, percentage in zip(indices, percentages):
        print("クラス:", class_idx[idx.item()], ", 確信度:", percentage.item())

    print('\n' + '=' * 50 + '\n')